In [1]:
import tensorflow as tf
from tensorflow import image as img
import os
import shutil, sys
import numpy as np
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import random
import keras
from keras.utils import plot_model
from keras.applications.mobilenet import MobileNet
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

Using TensorFlow backend.


In [2]:
from keras.models import load_model
model = load_model('keras_aug_4_15_12.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 6, 6, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              18875392  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_4 (Dense)              (None, 12)                780       
Total para

In [4]:
model.layers[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 196, 196, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 196, 196, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 196, 196, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 98, 98, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 98, 98, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 98, 98, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 49, 49, 128)       0         
__________

In [5]:
test_dir = './Plant-Seedlings-Classification/validation'
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(196, 196),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_generator,steps = nb_samples)

Found 360 images belonging to 12 classes.


In [6]:
predict.shape

(360, 12)

In [7]:
predict_labels = np.argmax(predict, axis=1)
val_labels = np.zeros(predict_labels.size, dtype=int)
for i in range(12):
    for j in range(30):
        val_labels[i*30+j] = int(i);

In [8]:
val_labels

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7

In [9]:
from sklearn.metrics import confusion_matrix 
confusion_matrix(val_labels, predict_labels)

array([[23,  0,  0,  0,  0,  0,  7,  0,  0,  0,  0,  0],
       [ 0, 30,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 30,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 29,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, 30,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 30,  0,  0,  0,  0,  0,  0],
       [ 3,  0,  0,  0,  0,  0, 27,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1, 29,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 30,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1, 29,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0, 29,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 30]])

In [10]:
for each_dir in os.listdir(test_dir):
    print(each_dir)

Sugar beet
Shepherds Purse
Scentless Mayweed
Charlock
Cleavers
Maize
Loose Silky-bent
Common wheat
Fat Hen
Small-flowered Cranesbill
Black-grass
Common Chickweed


In [11]:
test_generator.class_indices


{'Black-grass': 0,
 'Charlock': 1,
 'Cleavers': 2,
 'Common Chickweed': 3,
 'Common wheat': 4,
 'Fat Hen': 5,
 'Loose Silky-bent': 6,
 'Maize': 7,
 'Scentless Mayweed': 8,
 'Shepherds Purse': 9,
 'Small-flowered Cranesbill': 10,
 'Sugar beet': 11}

In [3]:
train_dir = './train/'
val_dir ='./Plant-Seedlings-Classification/validation/'

for each_dir in os.listdir(val_dir):
    #print(each_dir)
    for each_file in os.listdir(os.path.join(val_dir, each_dir)):
        path = os.path.join(train_dir, each_dir, each_file)
        #os.remove(path)
        #print(path)

FileNotFoundError: [Errno 2] No such file or directory: './Plant-Seedlings-Classification/validation/'

In [4]:
train2_dir = './train2'
val2_dir = './validation2'
data2 = ['Black-grass', 'Loose Silky-bent']

for each_dir in data2:
    src_path = os.path.join(train_dir, each_dir)
    dest_path = os.path.join(train2_dir, each_dir)
    shutil.copytree(src_path, dest_path)
    
for each_dir in data2:
    src_path = os.path.join(val_dir, each_dir)
    dest_path = os.path.join(val2_dir, each_dir)
    shutil.copytree(src_path, dest_path)
    



FileNotFoundError: [Errno 2] No such file or directory: './train/Black-grass'

In [7]:
# MODEL SMALL FOR BINARY CLASSIFICATION FOR CLASS 0 AND CLASS 6
#parameters
classes_2 = 2
image_size = 224
batch_size =30

In [3]:
model_mobilenet_2 = MobileNet(include_top=False, weights = 'imagenet',input_shape=(image_size,image_size,3), alpha=0.5)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
x = model_mobilenet_2.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes_2, activation='softmax')(x)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
model_2 = Model(inputs=model_mobilenet_2.input, outputs=x)

In [8]:
train2_dir = './train2'
val2_dir = './validation2'
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator_2 = train_datagen.flow_from_directory(
                    train2_dir,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator_2 = validation_datagen.flow_from_directory(
                        val2_dir,
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical')


Found 857 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [7]:
train_generator_2.class_indices

{'Black-grass': 0, 'Loose Silky-bent': 1}

In [10]:
model_2.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [12]:
history_2 = model_2.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=30,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2)

Epoch 1/30
 - 25s - loss: 0.4977 - acc: 0.7752 - val_loss: 0.6848 - val_acc: 0.6833
Epoch 2/30
 - 23s - loss: 0.5159 - acc: 0.7678 - val_loss: 0.7779 - val_acc: 0.6000
Epoch 3/30
 - 23s - loss: 0.4794 - acc: 0.7738 - val_loss: 0.8060 - val_acc: 0.5667
Epoch 4/30
 - 24s - loss: 0.4656 - acc: 0.7879 - val_loss: 0.7209 - val_acc: 0.6500
Epoch 5/30
 - 23s - loss: 0.4445 - acc: 0.7901 - val_loss: 0.6545 - val_acc: 0.7333
Epoch 6/30
 - 22s - loss: 0.4644 - acc: 0.7934 - val_loss: 0.7527 - val_acc: 0.6000
Epoch 7/30
 - 24s - loss: 0.4562 - acc: 0.8028 - val_loss: 0.5916 - val_acc: 0.7167
Epoch 8/30
 - 23s - loss: 0.3897 - acc: 0.8469 - val_loss: 0.6407 - val_acc: 0.6500
Epoch 9/30
 - 24s - loss: 0.4412 - acc: 0.8069 - val_loss: 0.6087 - val_acc: 0.7500
Epoch 10/30
 - 23s - loss: 0.3981 - acc: 0.8126 - val_loss: 0.5442 - val_acc: 0.7500
Epoch 11/30
 - 23s - loss: 0.3849 - acc: 0.8367 - val_loss: 0.5527 - val_acc: 0.7333
Epoch 12/30
 - 25s - loss: 0.3545 - acc: 0.8557 - val_loss: 0.4929 - val_a

In [13]:
model_2.save('2classifier-30-e-4.h5')

In [5]:
callbacks = [ModelCheckpoint(filepath='2classifier-30-e-5.h5', verbose=1),
ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=10, verbose=1),
#EarlyStopping(monitor='val_acc', patience=10, verbose=1),
CSVLogger('./10-metrics_model_2.csv')]

In [10]:
model_2 = keras.models.load_model('2classifier-30-e-4.h5')
history_2 = model_2.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=30,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    callbacks=callbacks)

Epoch 1/30
 - 26s - loss: 0.2020 - acc: 0.9117 - val_loss: 0.5727 - val_acc: 0.8167

Epoch 00001: saving model to 2classifier-30-e-5.h5
Epoch 2/30
 - 14s - loss: 0.1777 - acc: 0.9336 - val_loss: 0.5193 - val_acc: 0.8000

Epoch 00002: saving model to 2classifier-30-e-5.h5
Epoch 3/30
 - 23s - loss: 0.2214 - acc: 0.9198 - val_loss: 0.5595 - val_acc: 0.7500

Epoch 00003: saving model to 2classifier-30-e-5.h5
Epoch 4/30
 - 23s - loss: 0.2241 - acc: 0.9127 - val_loss: 0.7066 - val_acc: 0.7500

Epoch 00004: saving model to 2classifier-30-e-5.h5
Epoch 5/30
 - 23s - loss: 0.2047 - acc: 0.9117 - val_loss: 0.6747 - val_acc: 0.8000

Epoch 00005: saving model to 2classifier-30-e-5.h5
Epoch 6/30
 - 23s - loss: 0.1707 - acc: 0.9393 - val_loss: 0.9905 - val_acc: 0.7167

Epoch 00006: saving model to 2classifier-30-e-5.h5
Epoch 7/30
 - 23s - loss: 0.1895 - acc: 0.9278 - val_loss: 1.0373 - val_acc: 0.6833

Epoch 00007: saving model to 2classifier-30-e-5.h5
Epoch 8/30
 - 23s - loss: 0.1876 - acc: 0.9322 -

In [12]:
model_2.save('2clasifier_30_epochs.h5')

In [9]:
model_2 = keras.models.load_model('2clasifier_30_epochs.h5')
history_2 = model_2.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=30,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
 - 25s - loss: 0.5659 - acc: 0.7209 - val_loss: 0.8112 - val_acc: 0.6167
Epoch 2/30
 - 23s - loss: 0.5492 - acc: 0.7459 - val_loss: 0.8175 - val_acc: 0.6333
Epoch 3/30
 - 24s - loss: 0.5620 - acc: 0.7177 - val_loss: 0.8194 - val_acc: 0.6333
Epoch 4/30
 - 24s - loss: 0.5350 - acc: 0.7433 - val_loss: 0.8134 - val_acc: 0.6167
Epoch 5/30
 - 23s - loss: 0.5429 - acc: 0.7536 - val_loss: 0.8219 - val_acc: 0.6167
Epoch 6/30
 - 23s - loss: 0.5521 - acc: 0.7296 - val_loss: 0.8161 - val_acc: 0.6000
Epoch 7/30
 - 24s - loss: 0.5530 - acc: 0.7385 - val_loss: 0.7859 - val_acc: 0.6167
Epoch 8/30


KeyboardInterrupt: 

In [14]:
model_2.save('2clasifier_60_epochs.h5')
model_2.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])
model_2 = keras.models.load_model('2clasifier_60_epochs.h5')
history_2 = model_2.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=30,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2)

Epoch 1/30
 - 25s - loss: 0.4698 - acc: 0.7844 - val_loss: 0.7854 - val_acc: 0.6333
Epoch 2/30
 - 22s - loss: 0.4673 - acc: 0.7681 - val_loss: 0.7690 - val_acc: 0.6500
Epoch 3/30
 - 23s - loss: 0.5045 - acc: 0.7539 - val_loss: 0.7691 - val_acc: 0.6333
Epoch 4/30
 - 24s - loss: 0.4591 - acc: 0.7934 - val_loss: 0.7702 - val_acc: 0.6333
Epoch 5/30
 - 24s - loss: 0.4733 - acc: 0.7732 - val_loss: 0.7695 - val_acc: 0.6500
Epoch 6/30
 - 24s - loss: 0.4865 - acc: 0.7666 - val_loss: 0.7738 - val_acc: 0.6500
Epoch 7/30
 - 23s - loss: 0.4681 - acc: 0.7864 - val_loss: 0.7924 - val_acc: 0.6500
Epoch 8/30
 - 23s - loss: 0.4612 - acc: 0.7839 - val_loss: 0.7872 - val_acc: 0.6500
Epoch 9/30
 - 23s - loss: 0.4724 - acc: 0.7842 - val_loss: 0.7642 - val_acc: 0.6500
Epoch 10/30
 - 22s - loss: 0.4690 - acc: 0.7876 - val_loss: 0.7555 - val_acc: 0.6500
Epoch 11/30
 - 24s - loss: 0.4579 - acc: 0.8008 - val_loss: 0.7634 - val_acc: 0.6500
Epoch 12/30
 - 24s - loss: 0.4528 - acc: 0.7833 - val_loss: 0.7659 - val_a